# Extract Custom Fields from Your File

This notebook demonstrates how to use analyzers to extract custom fields from your input files.

## Prerequisites
1. Ensure Azure AI service is configured following [steps](../README.md#configure-azure-ai-service-resource)
2. Install the required packages to run the sample.

In [ ]:
%pip install -r ../requirements.txt

## Create Azure AI Content Understanding Client

> The [AzureContentUnderstandingClient](../python/content_understanding_client.py) is a utility class containing functions to interact with the Content Understanding API. Before the official release of the Content Understanding SDK, it can be regarded as a lightweight SDK. Fill the constant **AZURE_AI_ENDPOINT**, **AZURE_AI_API_VERSION**, **AZURE_AI_API_KEY** with the information from your Azure AI Service.

> ⚠️ Important:
You must update the code below to match your Azure authentication method.
Look for the `# IMPORTANT` comments and modify those sections accordingly.
If you skip this step, the sample may not run correctly.

> ⚠️ Note: Using a subscription key works, but using a token provider with Azure Active Directory (AAD) is much safer and is highly recommended for production environments.

In [ ]:
import logging
import json
import os
import sys
import uuid
from dotenv import load_dotenv
from azure.storage.blob import ContainerSasPermissions
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.ai.contentunderstanding.aio import ContentUnderstandingClient
from azure.ai.contentunderstanding.models import (
    ContentAnalyzer,
    ContentAnalyzerConfig,
    FieldSchema,
    FieldDefinition,
    FieldType,
    GenerationMethod,
    AnalysisMode,
    ProcessingLocation,
    SegmentationMode
)

# Add the parent directory to the Python path to import the sample_helper module
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'python'))
from extension.document_processor import DocumentProcessor
from extension.sample_helper import extract_operation_id_from_poller, PollerType, save_json_to_file

load_dotenv()
logging.basicConfig(level=logging.INFO)

endpoint = os.environ.get("AZURE_CONTENT_UNDERSTANDING_ENDPOINT")
# Return AzureKeyCredential if AZURE_CONTENT_UNDERSTANDING_KEY is set, otherwise DefaultAzureCredential
key = os.getenv("AZURE_CONTENT_UNDERSTANDING_KEY")
credential = AzureKeyCredential(key) if key else DefaultAzureCredential()
# Create the ContentUnderstandingClient
client = ContentUnderstandingClient(endpoint=endpoint, credential=credential)
print("✅ ContentUnderstandingClient created successfully")

try:
    processor = DocumentProcessor(client)
    print("✅ DocumentProcessor created successfully")
except Exception as e:
    print(f"❌ Failed to create DocumentProcessor: {e}")
    raise

## Analyzer Templates

This notebook demonstrates field extraction across multiple modalities using Azure AI Content Understanding. We'll walk through each modality step by step:

1. **Document Analysis** - Extract fields from invoices and receipts
2. **Audio Analysis** - Process call recordings and conversation audio  
3. **Video Analysis** - Analyze marketing videos and extract insights
4. **Image Analysis** - Extract information from charts and images

Each section will create an analyzer, process sample data, display results, and clean up the analyzer before moving to the next modality.

### Understanding Analyzer Templates and Schemas

Before we create our first analyzer, it's important to understand the structure and schema of analyzer templates. Custom analyzers in Azure AI Content Understanding are defined using JSON templates that specify what fields to extract and how to process the content.

**Key Schema Components:**

- **`baseAnalyzerId`**: This is crucial as it specifies which prebuilt analyzer to derive from (e.g., `prebuilt-documentAnalyzer`, `prebuilt-audioAnalyzer`, `prebuilt-videoAnalyzer`, `prebuilt-imageAnalyzer`, `prebuilt-callCenter`). This provides the foundation capabilities for your custom analyzer.

- **`fields`**: Define the specific data points you want to extract. Each field has:
  - **Field name**: The identifier for the extracted data (required and important for referencing results)
  - **Description**: Optional but helpful for documentation and understanding the field's purpose
  - **Method**: Can be `"extract"` (for extracting existing information from documents - currently only available for document analysis) or `"generate"` (for generating new insights using AI)

- **`method`**: The overall extraction approach - use `"extract"` for standard field extraction from documents or `"generate"` when you need AI to generate insights or summaries.

**Important Note**: The `"extract"` method is currently only available for document analysis (using `prebuilt-documentAnalyzer`). For other modalities like audio, video, and images, use the `"generate"` method.

Let's examine the invoice template to see these concepts in action:

The following is the schema for document modality used to extract fields from an invoice PDF. This analyzer identifies key invoice elements such as vendor information, amounts, dates, and line items.

In [ ]:
analyzer_template_path = '../analyzer_templates/invoice.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

Below is an example schema for the video modality.

In [ ]:
analyzer_template_path = '../analyzer_templates/marketing_video.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

# Document Analysis

Let's start with document analysis by extracting fields from invoices and receipts. This modality is excellent for processing structured documents and extracting key information like amounts, dates, vendor details, and line items.

## 1. Invoice Field Extraction

Let's extract fields from an invoice PDF. This analyzer identifies essential invoice elements such as vendor information, amounts, dates, and line items.

In [ ]:
analyzer_template_path = '../analyzer_templates/invoice.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Invoice Analyzer**

Now let's create the invoice analyzer and process our sample invoice:

In [ ]:
invoice_analyzer_id = "invoice-extraction-sample-" + str(uuid.uuid4())

invoice_analyzer = ContentAnalyzer(
    base_analyzer_id="prebuilt-documentAnalyzer",
    description="Sample invoice analyzer",
    field_schema=FieldSchema(
        fields={
            "VendorName": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.EXTRACT,
                description="Vendor issuing the invoice"
            ),
            "Items": FieldDefinition(
                type=FieldType.ARRAY,
                method=GenerationMethod.EXTRACT,
                items_property=FieldDefinition(
                    type=FieldType.OBJECT,
                    properties={
                        "Description": FieldDefinition(
                            type=FieldType.STRING,
                            method=GenerationMethod.EXTRACT,
                            description="Description of the item"
                        ),
                        "Amount": FieldDefinition(
                            type=FieldType.NUMBER,
                            method=GenerationMethod.EXTRACT,
                            description="Amount of the item"
                        )
                    }
                )
            )
        }
    ),
)
print(f"{json.dumps(invoice_analyzer.as_dict(), indent=2)}")
# Start the analyzer creation operation
poller = await client.content_analyzers.begin_create_or_replace(
    analyzer_id=invoice_analyzer_id,
    resource=invoice_analyzer,
)

# Extract operation ID from the poller
operation_id = extract_operation_id_from_poller(
    poller, PollerType.ANALYZER_CREATION
)
print(f"📋 Extracted creation operation ID: {operation_id}")

# Wait for the analyzer to be created
print(f"⏳ Waiting for analyzer creation to complete...")
await poller.result()
print(f"✅ Analyzer '{invoice_analyzer_id}' created successfully!")

In [ ]:
sample_file_path = '../data/invoice.pdf'

with open(sample_file_path, 'rb') as f:
    invoice_content = f.read()

# Begin document analysis operation
print(f"🔍 Starting document analysis with analyzer '{invoice_analyzer_id}'...")
analysis_poller = await client.content_analyzers.begin_analyze_binary(
    analyzer_id=invoice_analyzer_id,
    input=invoice_content,
    content_type="application/pdf",
)

# Wait for analysis completion
print(f"⏳ Waiting for document analysis to complete...")
analysis_result = await analysis_poller.result()
print(f"✅ Document analysis completed successfully!")

# Extract operation ID for get_result
analysis_operation_id = extract_operation_id_from_poller(
    analysis_poller, PollerType.ANALYZE_CALL
)
print(f"📋 Extracted analysis operation ID: {analysis_operation_id}")

# Get the analysis result using the operation ID
print(
    f"🔍 Getting analysis result using operation ID '{analysis_operation_id}'..."
)
operation_status = await client.content_analyzers.get_result(
    operation_id=analysis_operation_id,
)

print(f"✅ Analysis result retrieved successfully!")
print(f"   Operation ID: {operation_status.id}")
print(f"   Status: {operation_status.status}")

# The actual analysis result is in operation_status.result
operation_result = operation_status.result
if operation_result is None:
    print("⚠️  No analysis result available")

print(f"   Result contains {len(operation_result.contents)} contents")

# Save the analysis result to a file
saved_file_path = save_json_to_file(
    result=operation_result.as_dict(),
    filename_prefix="invoice_analyzers_get_result",
)
print(f"💾 Analysis result saved to: {saved_file_path}")

**Invoice Analysis Results**

Let's examine the extracted fields from the invoice:

In [ ]:
print(json.dumps(operation_result.as_dict(), indent=2))

**Clean Up Invoice Analyzer**

Clean up the analyzer to manage resources (in production, you would typically keep analyzers for reuse):

In [ ]:
await client.content_analyzers.delete(invoice_analyzer_id)

## 2. Invoice Field Extraction with Source Grounding

Now let's analyze the same invoice but with enhanced field source information and confidence scores. This provides additional context about where each extracted field was found in the document.

**Note**: This custom analyzer is for demo purposes only. In production, users should start from `prebuilt-invoice` for invoice processing.

**Key Feature**: This analyzer template uses `estimateFieldSourceAndConfidence: true` in the 'config', which enables the service to provide detailed information about:
- **Field source locations**: Exact coordinates and bounding boxes where each field was found
- **Confidence scores**: How confident the service is about each extracted field
- **Enhanced metadata**: Additional context about the extraction process

This is particularly useful for applications that need to verify extraction accuracy or provide visual feedback about where information was found in the source document.

In [ ]:
analyzer_template_path = '../analyzer_templates/invoice_field_source.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Invoice Field Source Analyzer**

Now let's create the enhanced analyzer and process the invoice with source grounding:

In [ ]:
invoice_source_analyzer_id = "invoice-source-extraction-sample-" + str(uuid.uuid4())

invoice_source_analyzer = ContentAnalyzer(
    base_analyzer_id="prebuilt-documentAnalyzer",
    description="Sample invoice analyzer",
    field_schema=FieldSchema(
        fields={
            "VendorName": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.EXTRACT,
                description="Vendor issuing the invoice"
            ),
            "Items": FieldDefinition(
                type=FieldType.ARRAY,
                method=GenerationMethod.EXTRACT,
                items_property=FieldDefinition(
                    type=FieldType.OBJECT,
                    properties={
                        "Description": FieldDefinition(
                            type=FieldType.STRING,
                            method=GenerationMethod.EXTRACT,
                            description="Description of the item"
                        ),
                        "Amount": FieldDefinition(
                            type=FieldType.NUMBER,
                            method=GenerationMethod.EXTRACT,
                            description="Amount of the item"
                        )
                    }
                )
            )
        }
    ),
)
print(f"{json.dumps(invoice_source_analyzer.as_dict(), indent=2)}")
# Start the analyzer creation operation
poller = await client.content_analyzers.begin_create_or_replace(
    analyzer_id=invoice_source_analyzer_id,
    resource=invoice_source_analyzer,
)

# Extract operation ID from the poller
operation_id = extract_operation_id_from_poller(
    poller, PollerType.ANALYZER_CREATION
)
print(f"📋 Extracted creation operation ID: {operation_id}")

# Wait for the analyzer to be created
print(f"⏳ Waiting for analyzer creation to complete...")
await poller.result()
print(f"✅ Analyzer '{invoice_source_analyzer_id}' created successfully!")

In [ ]:
sample_file_path = '../data/invoice.pdf'

with open(sample_file_path, 'rb') as f:
    invoice_source_content = f.read()

# Begin document analysis operation
print(f"🔍 Starting document analysis with analyzer '{invoice_source_analyzer_id}'...")
analysis_poller = await client.content_analyzers.begin_analyze_binary(
    analyzer_id=invoice_source_analyzer_id,
    input=invoice_source_content,
    content_type="application/pdf",
)

# Wait for analysis completion
print(f"⏳ Waiting for document analysis to complete...")
analysis_result = await analysis_poller.result()
print(f"✅ Document analysis completed successfully!")

# Extract operation ID for get_result
analysis_operation_id = extract_operation_id_from_poller(
    analysis_poller, PollerType.ANALYZE_CALL
)
print(f"📋 Extracted analysis operation ID: {analysis_operation_id}")

# Get the analysis result using the operation ID
print(
    f"🔍 Getting analysis result using operation ID '{analysis_operation_id}'..."
)
operation_status = await client.content_analyzers.get_result(
    operation_id=analysis_operation_id,
)

print(f"✅ Analysis result retrieved successfully!")
print(f"   Operation ID: {operation_status.id}")
print(f"   Status: {operation_status.status}")

# The actual analysis result is in operation_status.result
operation_result = operation_status.result
if operation_result is None:
    print("⚠️  No analysis result available")

print(f"   Result contains {len(operation_result.contents)} contents")

# Save the analysis result to a file
saved_file_path = save_json_to_file(
    result=operation_result.as_dict(),
    filename_prefix="invoice_source_content_analyzers_get_result",
)
print(f"💾 Analysis result saved to: {saved_file_path}")

**Invoice Field Source Analysis Results**

Let's review the enhanced results, which include detailed field source locations and confidence scores. Pay special attention to the `confidence` and `source` attributes for each extracted field. The `source` attribute provides the page number and bounding box coordinates for the extracted data, offering precise context for verification and visualization.

In [ ]:
print(json.dumps(operation_result.as_dict(), indent=2))

**Clean Up Invoice Field Source Analyzer**

Clean up the field source analyzer:

In [ ]:
print(f"🗑️  Deleting analyzer '{invoice_source_analyzer_id}' (demo cleanup)...")
await client.content_analyzers.delete(analyzer_id=invoice_source_analyzer_id)
print(f"✅ Analyzer '{invoice_source_analyzer_id}' deleted successfully!")

## 3. Receipt Field Extraction

Let's extract information from a receipt image. This demonstrates how the same Content Understanding service can handle different document types and formats.

**Receipt Analyzer Template**

Let's examine the receipt analyzer template:

In [ ]:
analyzer_template_path = '../analyzer_templates/receipt.json'
with open(analyzer_template_path, 'r') as f:
    template_content = json.load(f)
    print(json.dumps(template_content, indent=2))

**Create and Run Receipt Analyzer**

Now let's create the receipt analyzer and process our sample receipt:

In [ ]:
receipt_analyzer_id = "receipt-extraction-" + str(uuid.uuid4())

image_analyzer = ContentAnalyzer(
    base_analyzer_id="prebuilt-documentAnalyzer",
    description="Sample receipt analyzer",
    mode=AnalysisMode.STANDARD,
    processing_location=ProcessingLocation.GLOBAL,
    tags={"demo_type": "image_analysis"},
)

# Start the analyzer creation operation
poller = await client.content_analyzers.begin_create_or_replace(
    analyzer_id=receipt_analyzer_id,
    resource=image_analyzer,
)

# Extract operation ID from the poller
operation_id = extract_operation_id_from_poller(
    poller, PollerType.ANALYZER_CREATION
)
print(f"📋 Extracted creation operation ID: {operation_id}")

# Wait for the analyzer to be created
print(f"⏳ Waiting for analyzer creation to complete...")
await poller.result()
print(f"✅ Analyzer '{receipt_analyzer_id}' created successfully!")

In [ ]:
sample_file_path = '../data/receipt.png'

with open(sample_file_path, 'rb') as f:
    image_bytes: bytes = f.read()

print(f"🔍 Analyzing {sample_file_path} with prebuilt-documentAnalyzer...")

analysis_poller = await client.content_analyzers.begin_analyze_binary(
    analyzer_id=receipt_analyzer_id, 
    input=image_bytes,
    content_type="application/octet-stream")

# Wait for analysis completion
print(f"⏳ Waiting for document analysis to complete...")
analysis_result = await analysis_poller.result()
print(f"✅ Document analysis completed successfully!")

# Extract operation ID for get_result
analysis_operation_id = extract_operation_id_from_poller(
    analysis_poller, PollerType.ANALYZE_CALL
)
print(f"📋 Extracted analysis operation ID: {analysis_operation_id}")

# Get the analysis result using the operation ID
print(
    f"🔍 Getting analysis result using operation ID '{analysis_operation_id}'..."
)
operation_status = await client.content_analyzers.get_result(
    operation_id=analysis_operation_id,
)

print(f"✅ Analysis result retrieved successfully!")
print(f"   Operation ID: {operation_status.id}")
print(f"   Status: {operation_status.status}")

# The actual analysis result is in operation_status.result
operation_result = operation_status.result
if operation_result is None:
    print("⚠️  No analysis result available")
    
print(f"   Result contains {len(operation_result.contents)} contents")

# Save the analysis result to a file
saved_file_path = save_json_to_file(
    result=operation_result.as_dict(),
    filename_prefix="image_analyzer_get_result",
)
print(f"💾 Analysis result saved to: {saved_file_path}")

**Receipt Analysis Results**

Let's examine the extracted fields from the receipt:

In [ ]:
print(json.dumps(operation_result.as_dict(), indent=2))

**Clean Up Receipt Analyzer**

Clean up the receipt analyzer:

In [ ]:
print(f"🗑️  Deleting analyzer '{receipt_analyzer_id}' (demo cleanup)...")
await client.content_analyzers.delete(analyzer_id=receipt_analyzer_id)
print(f"✅ Analyzer '{receipt_analyzer_id}' deleted successfully!")

# Audio Analysis

Now let's move to audio analysis. This modality allows us to extract insights from audio files such as call recordings, including summaries, sentiment analysis, topic identification, and entity extraction.

## 4. Call Recording Analytics

Let's analyze a call center recording to extract insights such as summary, topics discussed, mentioned companies, and people involved in the conversation.

**Call Recording Analytics Template**

Let's examine the call recording analytics template:

In [ ]:
call_analyzer_id = "call-recording-analytics-" + str(uuid.uuid4())
# Create a custom analyzer using object model
print(f"🔧 Creating custom analyzer '{call_analyzer_id}'...")

call_analyzer = ContentAnalyzer(
    base_analyzer_id="prebuilt-callCenter",
    description="Sample call recording analytics",
    config=ContentAnalyzerConfig(
        return_details=True,
        locales=["en-US"]
    ),
    field_schema=FieldSchema(
        fields={
            "Summary": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.GENERATE,
                description="A one-paragraph summary"
            ),
            "Topics": FieldDefinition(
                type=FieldType.ARRAY,
                method=GenerationMethod.GENERATE,
                description="Top 5 topics mentioned",
                items_property=FieldDefinition(
                    type=FieldType.STRING
                )
            ),
            "Companies": FieldDefinition(
                type=FieldType.ARRAY,
                method=GenerationMethod.GENERATE,
                description="List of companies mentioned",
                items_property=FieldDefinition(
                    type=FieldType.STRING
                )
            ),
            "People": FieldDefinition(
                type=FieldType.ARRAY,
                method=GenerationMethod.GENERATE,
                description="List of people mentioned",
                items_property=FieldDefinition(
                    type=FieldType.OBJECT,
                    properties={
                        "Name": FieldDefinition(
                            type=FieldType.STRING,
                            description="Person's name"
                        ),
                        "Role": FieldDefinition(
                            type=FieldType.STRING,
                            description="Person's title/role"
                        )
                    }
                )
            ),
            "Sentiment": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.CLASSIFY,
                description="Overall sentiment",
                enum=[
                  "Positive",
                  "Neutral",
                  "Negative"
                ]
            ),
            "Categories": FieldDefinition(
                type=FieldType.ARRAY,
                method=GenerationMethod.CLASSIFY,
                description="List of relevant categories",
                items_property=FieldDefinition(
                  type=FieldType.STRING,
                  enum=[
                    "Agriculture",
                    "Business",
                    "Finance",
                    "Health",
                    "Insurance",
                    "Mining",
                    "Pharmaceutical",
                    "Retail",
                    "Technology",
                    "Transportation"
                  ]
                )
            )
        }
    )
)

# Start the analyzer creation operation
poller = await client.content_analyzers.begin_create_or_replace(
    analyzer_id=call_analyzer_id,
    resource=call_analyzer,
)

# Extract operation ID from the poller
operation_id = extract_operation_id_from_poller(
    poller, PollerType.ANALYZER_CREATION
)
print(f"📋 Extracted creation operation ID: {operation_id}")

# Wait for the analyzer to be created
print(f"⏳ Waiting for analyzer creation to complete...")
await poller.result()
print(f"✅ Analyzer '{call_analyzer_id}' created successfully!")

Create and run call recording analyzer

In [ ]:
# Read the sample MP3 file
sample_file_path = '../data/callCenterRecording.mp3'
print(f"📄 Reading audio file: {sample_file_path}")
with open(sample_file_path, 'rb') as f:
    audio_data = f.read()

# Begin document analysis operation
print(f"🔍 Starting document analysis with analyzer '{call_analyzer_id}'...")
analysis_poller = await client.content_analyzers.begin_analyze_binary(
    analyzer_id=call_analyzer_id,
    input=audio_data,
    content_type="application/octet-stream",
)

# Wait for analysis completion
print(f"⏳ Waiting for audio analysis to complete...")
analysis_result = await analysis_poller.result()
print(f"✅ Audio analysis completed successfully!")

# Extract operation ID for get_result
analysis_operation_id = extract_operation_id_from_poller(
    analysis_poller, PollerType.ANALYZE_CALL
)
print(f"📋 Extracted analysis operation ID: {analysis_operation_id}")

# Get the analysis result using the operation ID
print(
    f"🔍 Getting analysis result using operation ID '{analysis_operation_id}'..."
)
operation_status = await client.content_analyzers.get_result(
    operation_id=analysis_operation_id,
)

print(f"✅ Analysis result retrieved successfully!")
print(f"   Operation ID: {operation_status.id}")
print(f"   Status: {operation_status.status}")

# The actual analysis result is in operation_status.result
operation_result = operation_status.result
if operation_result is None:
    print("⚠️  No analysis result available")

print(f"   Result contains {len(operation_result.contents)} contents")

# Save the analysis result to a file
saved_file_path = save_json_to_file(
    result=operation_result.as_dict(),
    filename_prefix="call_analyzer_get_result",
)
print(f"💾 Analysis result saved to: {saved_file_path}")

Call recording analysis results

In [ ]:
print(json.dumps(operation_result.as_dict(), indent=2))

Clean up call recording analyzer

Note: In production environments, you would typically keep analyzers for reuse rather than deleting them

In [ ]:
print(f"🗑️  Deleting analyzer '{call_analyzer_id}' (demo cleanup)...")
await client.content_analyzers.delete(analyzer_id=call_analyzer_id)
print(f"✅ Analyzer '{call_analyzer_id}' deleted successfully!")

## 5. Conversational Audio Analytics

Let's analyze the same audio file but with a focus on conversational aspects like sentiment analysis and dialogue understanding.

Create and run conversational audio analyzer.

In [ ]:
conversation_analyzer_id = "conversational-audio-analytics-" + str(uuid.uuid4())
# Create a custom analyzer using object model
print(f"🔧 Creating custom analyzer '{conversation_analyzer_id}'...")

conversation_analyzer = ContentAnalyzer(
    description="Sample conversational audio analytics",
    base_analyzer_id="prebuilt-audioAnalyzer",
    config={
      "returnDetails": True,
      "locales": ["en-US"]
    },
    field_schema=FieldSchema(
      fields={
        "Summary": FieldDefinition(
          type=FieldType.STRING,
          method=GenerationMethod.GENERATE,
          description="A one-paragraph summary"
        ),
        "Sentiment": FieldDefinition(
          type=FieldType.STRING,
          method=GenerationMethod.CLASSIFY,
          description="Overall sentiment",
          enum=[
            "Positive",
            "Neutral",
            "Negative"
          ]
        )
      }
    )
)

# Create the analyzer
print(f"🔧 Creating custom analyzer '{conversation_analyzer_id}'...")
response = await client.content_analyzers.begin_create_or_replace(
    analyzer_id=conversation_analyzer_id,
    resource=conversation_analyzer
)

# Wait for the analyzer to be created
print(f"⏳ Waiting for analyzer creation to complete...")
await response.result()
print(f"✅ Analyzer '{conversation_analyzer_id}' created successfully!")

In [ ]:
# Read the sample file
audio_path = sample_file_path = '../data/callCenterRecording.mp3'
print(f"📄 Reading audio file: {audio_path}")
with open(audio_path, "rb") as audio_file:
    audio_content = audio_file.read()

# Begin audio analysis operation
print(f"🔍 Starting audio analysis with analyzer '{conversation_analyzer_id}'...")
analysis_poller = await client.content_analyzers.begin_analyze_binary(
    analyzer_id=conversation_analyzer_id,
    input=audio_content,
    content_type="application/octet-stream",
)

# Wait for analysis completion
print(f"⏳ Waiting for audio analysis to complete...")
analysis_result = await analysis_poller.result()
print(f"✅ Audio analysis completed successfully!")

# Extract operation ID for get_result
analysis_operation_id = extract_operation_id_from_poller(
    analysis_poller, PollerType.ANALYZE_CALL
)
print(f"📋 Extracted analysis operation ID: {analysis_operation_id}")

# Get the analysis result using the operation ID
print(
    f"🔍 Getting analysis result using operation ID '{analysis_operation_id}'..."
)
operation_status = await client.content_analyzers.get_result(
    operation_id=analysis_operation_id,
)

print(f"✅ Analysis result retrieved successfully!")
print(f"   Operation ID: {operation_status.id}")
print(f"   Status: {operation_status.status}")

# The actual analysis result is in operation_status.result
operation_result = operation_status.result
if operation_result is None:
    print("⚠️  No analysis result available")

print(f"   Result contains {len(operation_result.contents)} contents")

# Save the analysis result to a file
saved_file_path = save_json_to_file(
    result=operation_result.as_dict(),
    filename_prefix="conversation_analyzers_get_result",
)
print(f"💾 Analysis result saved to: {saved_file_path}")

Conversational audio analysis results

In [ ]:
print(json.dumps(operation_result.as_dict(), indent=2))

Clean up conversational audio analyzer

Note: In production environments, you would typically keep analyzers for reuse rather than deleting them

In [ ]:
print(f"🗑️  Deleting analyzer '{conversation_analyzer_id}' (demo cleanup)...")
await client.content_analyzers.delete(analyzer_id=conversation_analyzer_id)
print(f"✅ Analyzer '{conversation_analyzer_id}' deleted successfully!")

# Video Analysis

Now let's explore video analysis capabilities. This modality can extract insights from video content including descriptions, sentiment analysis, and key scene identification.

## 6. Marketing Video Analysis

Let's analyze a marketing video to extract descriptions, sentiment, and key insights that could be valuable for content understanding and marketing analytics.

Content Understanding offers three ways to slice a video, letting you get the output you need for whole videos or short clips. You can use these options by setting the `segmentationMode` property on a custom analyzer.
- Whole-video – `"segmentationMode": "noSegmentation"` The service treats the entire video file as a single segment and extracts metadata across its full duration.  
  Example:
    - Compliance checks that look for specific brand-safety issues anywhere in an ad
    - full-length descriptive summaries
- Automatic segmentation – `"segmentationMode": "auto"` The service analyzes the timeline and breaks it up for you. Groups successive shots into coherent scenes, capped at one minute each.  
  Example:
    - Create storyboards from a show
    - Inserting mid-roll ads at logical pauses.
- Custom segmentation – `"segmentationMode": "custom"` You describe the logic in natural language and the model creates segments to match. Set `segmentationDefinition` with a string describing how you'd like the video to be segmented. Custom allows segments of varying length from seconds to minutes depending on the prompt.  
  Example:
    - Break a news broadcast up into stories.

### 6-1 Analyze without Segmentation

In this example, we analyze a marketing video without segmentation.
- Please set `segmentationMode` to `noSegmentation` in the analyzer schema `config` to process the entire video as one segment.

Create and run marketing video analyzer

In [ ]:
video_analyzer_id = "marketing-video-analytics-" + str(uuid.uuid4())

# Create a custom analyzer using object model
print(f"🔧 Creating custom analyzer '{video_analyzer_id}'...")
video_content_analyzer = ContentAnalyzer(
    description="Sample marketing video analytics",
    base_analyzer_id="prebuilt-videoAnalyzer",
    config=ContentAnalyzerConfig(
        return_details=True,
        segmentation_mode=SegmentationMode.NO_SEGMENTATION
    ),
    field_schema=FieldSchema(
        fields={
            "Description": FieldDefinition(
                type=FieldType.STRING,
                description="Detailed summary of the video segment, focusing on product characteristics, lighting, and color palette."
            ),
            "Sentiment": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.CLASSIFY,
                enum=[
                    "Positive",
                    "Neutral",
                    "Negative"
                ]
            )
        }
    )
)

# Start the analyzer creation operation
poller = await client.content_analyzers.begin_create_or_replace(
    analyzer_id=video_analyzer_id,
    resource=video_content_analyzer,
)

# Extract operation ID from the poller
operation_id = extract_operation_id_from_poller(
    poller, PollerType.ANALYZER_CREATION
)
print(f"📋 Extracted creation operation ID: {operation_id}")

# Wait for the analyzer to be created
print(f"⏳ Waiting for analyzer creation to complete...")
await poller.result()
print(f"✅ Analyzer '{video_analyzer_id}' created successfully!")

In [ ]:
# Read the sample video file
video_path = '../data/FlightSimulator.mp4'
print(f"📄 Reading video file: {video_path}")
with open(video_path, "rb") as video_file:
    video_content = video_file.read()

# Begin video analysis operation
print(f"🔍 Starting video analysis with analyzer '{video_analyzer_id}'...")
analysis_poller = await client.content_analyzers.begin_analyze_binary(
    analyzer_id=video_analyzer_id,
    input=video_content,
    content_type="application/octet-stream",
)

# Wait for analysis completion
print(f"⏳ Waiting for video analysis to complete...")
analysis_result = await analysis_poller.result()
print(f"✅ Video analysis completed successfully!")

# Extract operation ID for get_result
analysis_operation_id = extract_operation_id_from_poller(
    analysis_poller, PollerType.ANALYZE_CALL
)
print(f"📋 Extracted analysis operation ID: {analysis_operation_id}")

# Get the analysis result using the operation ID
print(
    f"🔍 Getting analysis result using operation ID '{analysis_operation_id}'..."
)
operation_status = await client.content_analyzers.get_result(
    operation_id=analysis_operation_id,
)

print(f"✅ Analysis result retrieved successfully!")
print(f"   Operation ID: {operation_status.id}")
print(f"   Status: {operation_status.status}")

# The actual analysis result is in operation_status.result
operation_result = operation_status.result
if operation_result is None:
    print("⚠️  No analysis result available")

print(f"   Result contains {len(operation_result.contents)} contents")

# Save the analysis result to a file
saved_file_path = save_json_to_file(
    result=operation_result.as_dict(),
    filename_prefix="video_content_analyzers_get_result",
)
print(f"💾 Analysis result saved to: {saved_file_path}")

Marketing video analysis result
- The result is generated from the content of the entire video.

In [ ]:
print(json.dumps(operation_result.as_dict(), indent=2))

Clean up marketing video analyzer

Note: In production environments, you would typically keep analyzers for reuse rather than deleting them

In [ ]:
print(f"🗑️  Deleting analyzer '{video_analyzer_id}' (demo cleanup)...")
await client.content_analyzers.delete(analyzer_id=video_analyzer_id)
print(f"✅ Analyzer '{video_analyzer_id}' deleted successfully!")

### 6-2 Analyze With Automatic Segmentation

In this example, we use automatic segmentation for marketing video analytics.  
- Please set `segmentationMode` to `auto` in the analyzer schema `config` to enable automatic segmentation.

Create and run marketing video analyzer

In [ ]:
video_analyzer_id = "marketing-video-analytics-" + str(uuid.uuid4())

# Create a custom analyzer using object model
print(f"🔧 Creating custom analyzer '{video_analyzer_id}'...")

video_content_analyzer = ContentAnalyzer(
    description="Sample marketing video analytics",
    base_analyzer_id="prebuilt-videoAnalyzer",
    config=ContentAnalyzerConfig(
        return_details=True,
        segmentation_mode=SegmentationMode.NO_SEGMENTATION
    ),
    field_schema=FieldSchema(
        fields={
            "Segments": FieldDefinition(
                type=FieldType.ARRAY,
                items_property=FieldDefinition(
                    type=FieldType.OBJECT,
                    properties={
                        "Description": FieldDefinition(
                            type=FieldType.STRING,
                            description="Detailed summary of the video segment, focusing on product characteristics, lighting, and color palette."
                        ),
                        "Sentiment": FieldDefinition(
                            type=FieldType.STRING,
                            method=GenerationMethod.CLASSIFY,
                            enum=[
                                "Positive",
                                "Neutral",
                                "Negative"
                            ]
                        )
                    }
                )
            )
        }
    )
)

# Start the analyzer creation operation
poller = await client.content_analyzers.begin_create_or_replace(
    analyzer_id=video_analyzer_id,
    resource=video_content_analyzer,
)

# Extract operation ID from the poller
operation_id = extract_operation_id_from_poller(
    poller, PollerType.ANALYZER_CREATION
)
print(f"📋 Extracted creation operation ID: {operation_id}")

# Wait for the analyzer to be created
print(f"⏳ Waiting for analyzer creation to complete...")
await poller.result()
print(f"✅ Analyzer '{video_analyzer_id}' created successfully!")

In [ ]:
video_file_path = '../data/FlightSimulator.mp4'

print(f"📄 Reading video file: {video_path}")
with open(video_path, "rb") as video_file:
    video_content = video_file.read()

# Begin video analysis operation
print(f"🔍 Starting video analysis with analyzer '{video_analyzer_id}'...")
analysis_poller = await client.content_analyzers.begin_analyze_binary(
    analyzer_id=video_analyzer_id,
    input=video_content,
    content_type="application/octet-stream",
)

# Wait for analysis completion
print(f"⏳ Waiting for video analysis to complete...")
analysis_result = await analysis_poller.result()
print(f"✅ Video analysis completed successfully!")

# Extract operation ID for get_result
analysis_operation_id = extract_operation_id_from_poller(
    analysis_poller, PollerType.ANALYZE_CALL
)
print(f"📋 Extracted analysis operation ID: {analysis_operation_id}")

# Get the analysis result using the operation ID
print(
    f"🔍 Getting analysis result using operation ID '{analysis_operation_id}'..."
)
operation_status = await client.content_analyzers.get_result(
    operation_id=analysis_operation_id,
)

print(f"✅ Analysis result retrieved successfully!")
print(f"   Operation ID: {operation_status.id}")
print(f"   Status: {operation_status.status}")

# The actual analysis result is in operation_status.result
operation_result = operation_status.result
if operation_result is None:
    print("⚠️  No analysis result available")

print(f"   Result contains {len(operation_result.contents)} contents")

# Save the analysis result to a file
saved_file_path = save_json_to_file(
    result=operation_result.as_dict(),
    filename_prefix="video_content_analyzers_get_result",
)
print(f"💾 Analysis result saved to: {saved_file_path}")

Marketing video analysis result
- The output includes automatically segmented clips with descriptions in the markdown content.  
- The analyzer generates the fields defined in the schema separately for each segment.

In [ ]:
print(json.dumps(operation_result.as_dict(), indent=2))

Clean up marketing video analyzer

Note: In production environments, you would typically keep analyzers for reuse rather than deleting them

In [ ]:
print(f"🗑️  Deleting analyzer '{video_analyzer_id}' (demo cleanup)...")
await client.content_analyzers.delete(analyzer_id=video_analyzer_id)
print(f"✅ Analyzer '{video_analyzer_id}' deleted successfully!")

### 6-3 Analyze With Custom Segmentation

In this example, we use custom segmentation for marketing video analytics.  
- Please set `segmentationMode` to `custom`.  
- Provide a `segmentationDefinition` string describing how you would like the video to be segmented.

Create and run marketing video analyzer

In [ ]:
video_analyzer_id = "marketing-video-analytics-" + str(uuid.uuid4())

# Create a custom analyzer using object model
print(f"🔧 Creating custom analyzer '{video_analyzer_id}'...")

video_content_analyzer = ContentAnalyzer(
    description="Sample marketing video analytics",
    base_analyzer_id="prebuilt-videoAnalyzer",
    config=ContentAnalyzerConfig(
        return_details=True,
        segmentation_mode=SegmentationMode.NO_SEGMENTATION,
        segmentation_definition="Segment the video at each clear narrative or visual transition that introduces a new marketing message, speaker, or brand moment. Segments should begin when there is a change in speaker, a shift in visual theme (e.g., logos, product shots, data center views, simulation footage, aircraft scenes), or the introduction of a new key message (e.g., quality of data, scale of infrastructure, customer benefit, real-world aviation use). Each segment should capture one distinct marketing idea or value point, ending when the focus transitions to the next theme."
    ),
    field_schema=FieldSchema(
        fields={
            "Segments": FieldDefinition(
                type=FieldType.ARRAY,
                items_property=FieldDefinition(
                    type=FieldType.OBJECT,
                    properties={
                        "Description": FieldDefinition(
                            type=FieldType.STRING,
                            description="Detailed summary of the video segment, focusing on product characteristics, lighting, and color palette."
                        ),
                        "Sentiment": FieldDefinition(
                            type=FieldType.STRING,
                            method=GenerationMethod.CLASSIFY,
                            enum=[
                                "Positive",
                                "Neutral",
                                "Negative"
                            ]
                        )
                    }
                )
            )
        }
    )
)

# Start the analyzer creation operation
poller = await client.content_analyzers.begin_create_or_replace(
    analyzer_id=video_analyzer_id,
    resource=video_content_analyzer,
)

# Extract operation ID from the poller
operation_id = extract_operation_id_from_poller(
    poller, PollerType.ANALYZER_CREATION
)
print(f"📋 Extracted creation operation ID: {operation_id}")

# Wait for the analyzer to be created
print(f"⏳ Waiting for analyzer creation to complete...")
await poller.result()
print(f"✅ Analyzer '{video_analyzer_id}' created successfully!")

In [ ]:
video_file_path = '../data/FlightSimulator.mp4'

print(f"📄 Reading video file: {video_path}")
with open(video_path, "rb") as video_file:
    video_content = video_file.read()

# Begin video analysis operation
print(f"🔍 Starting video analysis with analyzer '{video_analyzer_id}'...")
analysis_poller = await client.content_analyzers.begin_analyze_binary(
    analyzer_id=video_analyzer_id,
    input=video_content,
    content_type="application/octet-stream",
)

# Wait for analysis completion
print(f"⏳ Waiting for video analysis to complete...")
analysis_result = await analysis_poller.result()
print(f"✅ Video analysis completed successfully!")

# Extract operation ID for get_result
analysis_operation_id = extract_operation_id_from_poller(
    analysis_poller, PollerType.ANALYZE_CALL
)
print(f"📋 Extracted analysis operation ID: {analysis_operation_id}")

# Get the analysis result using the operation ID
print(
    f"🔍 Getting analysis result using operation ID '{analysis_operation_id}'..."
)
operation_status = await client.content_analyzers.get_result(
    operation_id=analysis_operation_id,
)

print(f"✅ Analysis result retrieved successfully!")
print(f"   Operation ID: {operation_status.id}")
print(f"   Status: {operation_status.status}")

# The actual analysis result is in operation_status.result
operation_result = operation_status.result
if operation_result is None:
    print("⚠️  No analysis result available")

print(f"   Result contains {len(operation_result.contents)} contents")

# Save the analysis result to a file
saved_file_path = save_json_to_file(
    result=operation_result.as_dict(),
    filename_prefix="video_content_analyzers_get_result",
)
print(f"💾 Analysis result saved to: {saved_file_path}")

Marketing video analysis result
- The video is segmented according to your custom definition, with segment descriptions included in the markdown content.  
- The segmentation may differ from automatic segmentation results.  
- The analyzer generates the fields defined in the schema separately for each segment.

In [ ]:
print(json.dumps(operation_result.as_dict(), indent=2))

Clean up marketing video analyzer

Note: In production environments, you would typically keep analyzers for reuse rather than deleting them

In [ ]:
print(f"🗑️  Deleting analyzer '{video_analyzer_id}' (demo cleanup)...")
await client.content_analyzers.delete(analyzer_id=video_analyzer_id)
print(f"✅ Analyzer '{video_analyzer_id}' deleted successfully!")

# Image Analysis

Finally, let's explore image analysis capabilities. This modality can extract information from charts, diagrams, and other visual content.

## 7. Chart and Image Analysis

Let's analyze a chart image to extract data points, trends, and insights from visual data representations. \
Create and run chart image analyzer:

In [ ]:
chart_analyzer_id = "chart-analysis-" + str(uuid.uuid4())

# Create a custom analyzer using object model
print(f"🔧 Creating custom analyzer '{chart_analyzer_id}'...")

chart_content_analyzer = ContentAnalyzer(
    base_analyzer_id="prebuilt-imageAnalyzer",
    description="Extract detailed structured information from charts and diagrams.",
    config=ContentAnalyzerConfig(
        return_details=False,
    ),
    field_schema=FieldSchema(
        name="ChartAndDiagram",
        description="Structured information from charts and diagrams.",
        fields={
            "Title": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.GENERATE,
                description="Verbatim title of the chart."
            ),
            "ChartType": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.CLASSIFY,
                description="The type of chart.",
                enum=[
                    "area",
                    "bar",
                    "box",
                    "bubble",
                    "candlestick",
                    "funnel",
                    "heatmap",
                    "histogram",
                    "line",
                    "pie",
                    "radar",
                    "rings",
                    "rose",
                    "treemap"
                ],
                enum_descriptions={
                    "histogram": "Continuous values on the x-axis, which distinguishes it from bar.",
                    "rose": "In contrast to pie charts, the sectors are of equal angles and differ in how far each sector extends from the center of the circle."
                },
            ),
            "TopicKeywords": FieldDefinition(
                type=FieldType.ARRAY,
                method=GenerationMethod.GENERATE,
                description="Relevant topics associated with the chart, used for tagging.",
                items_property=FieldDefinition(
                    type=FieldType.STRING,
                    method=GenerationMethod.GENERATE,
                    examples=[
                        "Business and finance",
                        "Arts and culture",
                        "Education and academics"
                    ]
                )
            ),
            "DetailedDescription": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.GENERATE,
                description="Detailed description of the chart or diagram, not leaving out any key information. Include numbers, trends, and other details."
            ),
            "Summary": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.GENERATE,
                description="Detailed summary of the chart, including highlights and takeaways."
            ),
            "MarkdownDataTable": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.GENERATE,
                description="Underlying data of the chart in tabular markdown format. Give markdown output with valid syntax and accurate numbers, and fill any uncertain values with empty cells. If not applicable, output an empty string."
            ),
            "AxisTitles": FieldDefinition(
                type=FieldType.OBJECT,
                method=GenerationMethod.GENERATE,
                description="Titles of the axes.",
                properties={
                    "xAxisTitle": FieldDefinition(
                        type=FieldType.STRING,
                        method=GenerationMethod.GENERATE,
                        description="Title of the x-axis."
                    ),
                    "yAxisTitle": FieldDefinition(
                        type=FieldType.STRING,
                        method=GenerationMethod.GENERATE,
                        description="Title of the y-axis."
                    )
                }
            ),
            "FootnotesAndAnnotations": FieldDefinition(
                type=FieldType.STRING,
                method=GenerationMethod.GENERATE,
                description="All footnotes and textual annotations in the chart or diagram."
            )
        },
    ),
)

# Start the analyzer creation operation
poller = await client.content_analyzers.begin_create_or_replace(
    analyzer_id=chart_analyzer_id,
    resource=chart_content_analyzer,
)

# Extract operation ID from the poller
operation_id = extract_operation_id_from_poller(
    poller, PollerType.ANALYZER_CREATION
)
print(f"📋 Extracted creation operation ID: {operation_id}")

# Wait for the analyzer to be created
print(f"⏳ Waiting for analyzer creation to complete...")
await poller.result()
print(f"✅ Analyzer '{chart_analyzer_id}' created successfully!")

In [ ]:
# Failed
# Error Code: analysis_result = await analysis_poller.result()
# Inner error: {
#     "code": "ServiceUnavailable",
#     "message": "The service is currently unavailable. Please try again later."
# }

sample_file_path = '../data/pieChart.jpg'
print(f"📄 Reading document file: {sample_file_path}")
with open(sample_file_path, "rb") as f:
    chart_content = f.read()

# Check if this is a Git LFS pointer file
analysis_poller = await client.content_analyzers.begin_analyze_binary(
    analyzer_id=chart_analyzer_id,
    input=chart_content,
    content_type="application/octet-stream",
)

# Wait for analysis completion
print(f"⏳ Waiting for document analysis to complete...")
analysis_result = await analysis_poller.result()
print(f"✅ Document analysis completed successfully!")

# Extract operation ID for get_result
analysis_operation_id = extract_operation_id_from_poller(
    analysis_poller, PollerType.ANALYZE_CALL
)
print(f"📋 Extracted analysis operation ID: {analysis_operation_id}")

# Get the analysis result using the operation ID
print(
    f"🔍 Getting analysis result using operation ID '{analysis_operation_id}'..."
)
operation_status = await client.content_analyzers.get_result(
    operation_id=analysis_operation_id,
)

print(f"✅ Analysis result retrieved successfully!")
print(f"   Operation ID: {operation_status.id}")
print(f"   Status: {operation_status.status}")

# The actual analysis result is in operation_status.result
operation_result = operation_status.result
if operation_result is None:
    print("⚠️  No analysis result available")

print(f"   Result contains {len(operation_result.contents)} contents")

# Save the analysis result to a file
saved_file_path = save_json_to_file(
    result=operation_result.as_dict(),
    filename_prefix="content_analyzers_get_result",
)
print(f"💾 Analysis result saved to: {saved_file_path}")

# Save the analysis result to a file
saved_file_path = save_json_to_file(
    result=operation_result.as_dict(),
    filename_prefix="content_analyzers_get_result",
)
print(f"💾 Analysis result saved to: {saved_file_path}")

Chart analysis result:

In [ ]:
print(json.dumps(operation_result.as_dict(), indent=2))

Clean up chart image analyzer

Note: In production environments, you would typically keep analyzers for reuse rather than deleting them

In [ ]:
print(f"🗑️  Deleting analyzer '{chart_analyzer_id}' (demo cleanup)...")
await client.content_analyzers.delete(analyzer_id=chart_analyzer_id)
print(f"✅ Analyzer '{chart_analyzer_id}' deleted successfully!")

# Summary

🎉 **Congratulations!** You've successfully explored all the major modalities of Azure AI Content Understanding:

✅ **Document Analysis**: Extracted fields from invoices and receipts  
✅ **Audio Analysis**: Analyzed call recordings and conversational audio  
✅ **Video Analysis**: Processed marketing videos for insights  
✅ **Image Analysis**: Extracted information from charts and visual content

## Key Takeaways

- **Multi-modal capabilities**: Content Understanding can process documents, audio, video, and images
- **Customizable templates**: Each analyzer template is designed for specific use cases but can be customized
- **Automatic cleanup**: Each analyzer was automatically cleaned up after use to manage resources
- **Structured output**: All results are returned in consistent JSON format for easy integration

## Next Steps

- Explore the analyzer templates in the `../analyzer_templates/` directory
- Modify existing templates or create custom ones for your specific use cases
- Check out other notebooks in this repository for advanced scenarios
- Visit the [Azure AI Content Understanding documentation](https://docs.microsoft.com/azure/ai-services/content-understanding/) for more information